# Prediction Comparison for UNet Versions

This plots the model segmentation predictions versus the ground truth.

We need to compare the model outputs based on doing the **up sampling operation** via:
+ [UpSampling2D](https://keras.io/layers/convolutional/#upsampling2d)
   ```
   up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
   ```
+ [Conv2DTranspose](https://keras.io/layers/convolutional/#conv2dtranspose)
   ```
   up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=-1)
   ```
   
Intel has optimized the operations for Conv2DTranspose but not for UpSampling2D. If we can prove that the two produce similar results, then we can recommend the Intel optimization as a viable substitution.

In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def dice(im1, im2):
    
    '''
    Do we match the binary version of the prediction mask or the probability version?
    '''
    
    im1 = np.asarray(im1)
    im2 = np.asarray(im2)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = im1*im2

    smoothing_delta = 1e-7
    
    return (2. * intersection.sum() + smoothing_delta) / (im1.sum() + im2.sum() + smoothing_delta)

# Loading the prediction masks and the ground truth data

The *pred.npy* files were generated by running:
```
numactl -p 1 python train.py 50 5 0
```

In [3]:
msk_upsampling = np.load('msks_pred_upsampling.npy')
msk_transposed = np.load('msks_pred_dropout.npy') #transposed.npy')
dir1 = '/mnt/data/medical/brats/all_slices/Results/'
test = np.load(dir1+'msks_test.npy')
imgs = np.load(dir1+'imgs_test.npy')

IOError: [Errno 2] No such file or directory: 'msks_pred_upsampling.npy'

In [ ]:
# Preprocess the segmentation masks for the ground truth (mode=1)
test[:,:,:,0] = test[:,:,:,0]+test[:,:,:,1]+test[:,:,:,2]+test[:,:,:,3]

In [ ]:
def comparePredictions(img):
    
    plt.figure(figsize=(14,14))
    plt.subplot(1,4,1)
    plt.imshow(msk_upsampling[img,:,:,0], cmap='bone')
    plt.title('Prediction UpSampling')
    plt.subplot(1,4,2)
    plt.imshow(msk_transposed[img,:,:,0], cmap='bone')
    plt.yticks([])
    plt.title('Prediction Transposed')
    plt.subplot(1,4,3)
    plt.imshow(test[img,:,:,0], cmap='bone')
    plt.yticks([])
    plt.title('Ground truth');
    plt.subplot(1,4,4)
    plt.imshow(imgs[img,:,:,0]);
    plt.imshow(test[img,:,:,0], alpha=0.5);
    plt.yticks([])
    plt.title('Image with mask')
    
    # Calculate the Dice coefficient for the two versions
    msk_binaryU = msk_upsampling[img,:,:,0]
    print ('Dice coefficient for upsampling {}'.format(dice(msk_binaryU, test[img,:,:,0])))
    
    msk_binaryT = msk_transposed[img,:,:,0]
    print ('Dice coefficient for transposed {}'.format(dice(msk_binaryT, test[img,:,:,0])))
    

In [ ]:
comparePredictions(5821)

In [ ]:
comparePredictions(400)

In [ ]:
comparePredictions(520)

In [ ]:
comparePredictions(1333)

In [ ]:
comparePredictions(2540)

In [ ]:
comparePredictions(5533)

In [ ]:
comparePredictions(2121)

In [ ]:
comparePredictions(4444)

In [ ]:
comparePredictions(3334)

In [ ]:
comparePredictions(2560)

In [ ]:
comparePredictions(10)

In [ ]:
comparePredictions(88)

In [ ]:
comparePredictions(150)

In [ ]:
comparePredictions(231)

In [ ]:
dice_transposed = np.zeros(test.shape[0])
dice_upsampling = np.zeros(test.shape[0])
mass = np.zeros(test.shape[0])

for i in range(test.shape[0]):
    
    dice_transposed[i] = dice(msk_transposed[i, :, :, 0], test[i, :, :, 0])
    dice_upsampling[i] = dice(msk_upsampling[i, :, :, 0], test[i, :, :, 0])
    mass[i] = np.sum(test[i, :, :, 0]) # The amount of tumor pixels in the image
    

In [ ]:
plt.figure(figsize=(12,6));
plt.hist([dice_transposed, dice_upsampling]);
plt.legend(['Conv2DTranspose', 'UpSampling2D'], fontsize=18);
plt.title('Comparing Performance of the Two Models', fontsize=18);
plt.xlabel('Dice Coefficient', fontsize=14);
plt.ylabel('Number of occurrences', fontsize=14);

In [ ]:
plt.figure(figsize=(12,6));
plt.hist(dice_transposed - dice_upsampling, 50);
plt.title('Dice Difference (transposed - upsampling)', fontsize=18);
plt.text(0.2, 1500, r'Transposed better $\longrightarrow$', fontsize=18);
plt.text(-.8, 1500, r'$\longleftarrow$ Upsampling better', fontsize=18);
plt.xlabel('Difference in Dice Coefficients', fontsize=14);
plt.ylabel('Number of occurrences', fontsize=14)
plt.grid();

# Kolmogorov-Smirnoff (KS) Test

Perform a [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Two-sample_Kolmogorov.E2.80.93Smirnov_test) two sample test. The null hypothesis is that the two data samples come from the same distribution. 

In [ ]:
from scipy.stats import ks_2samp

ks_2samp(dice_transposed, dice_upsampling)

# Two-sided t-Test

Check if the difference between the Dice measures for each prediction is significantly different from 0.

In [ ]:
from scipy.stats import ttest_1samp

ttest_1samp(dice_transposed-dice_upsampling, 0)

In [ ]:
np.mean(dice_transposed)

In [ ]:
np.mean(dice_upsampling)

# Where are most of the poor predictions coming from?

It looks like when the Dice coefficient is low that typically there is not enough of a segmentation mask to make a sufficiently good prediction. To measure this, I calculated the "mass" of the segmentation map (i.e. just counted how many pixels were in the mask). Low dice scores were highly correlated with smaller segmentation masks.

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(mass[np.where(dice_upsampling < 0.25)], bins=10, alpha=0.8);
plt.hist(mass[np.where(dice_upsampling >= 0.25)], bins=10, alpha=0.8);
plt.title('Poor predictions are correlated with small predictions masks', fontsize=14);
plt.ylabel('Number of occurrences', fontsize=12);
plt.xlabel('Number of pixels in ground truth segmentation mask', fontsize=12);
plt.legend(['\lt 0.25', '\ge 0.25'], title='Dice coefficient', fontsize=14);